<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/GS%20-%20Guardar%20Vocabulario%20-%20Senado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f base_formateada_senado_snowball.csv
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f partidos.csv

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
base_formateada_senado_snowball.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/2.55k [00:00<?, ?B/s]
100% 2.55k/2.55k [00:00<00:00, 6.37MB/s]


In [4]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [8]:
import pandas as pd
base=pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-8-ae912bff9a98>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
base.head(2)

,Unnamed: 0.1,Unnamed: 0,Nombre/Cargo,Discurso,Cargo,Nombre,Link,Legislatura,tipo,fecha,fecha_formato,nombre,apellidos,procedLiteral,procedLugar,grupoNombre,tokens
0,0,0,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,Se abre la sesión.,PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10,JUNTA PREPARATORIA Y SESIÓN CONSTITUTIVACORTES...,13 de diciembre de 2011,2011-12-13,GUSTAVO,ALCALDE SÁNCHEZ,Designado,Designado: Cortes de Aragón,GPP,['abre']
1,1,1,EL señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,En cumplimiento de lo establecido en el aparta...,EL PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10,JUNTA PREPARATORIA Y SESIÓN CONSTITUTIVACORTES...,13 de diciembre de 2011,2011-12-13,GUSTAVO,ALCALDE SÁNCHEZ,Designado,Designado: Cortes de Aragón,GPP,"['cumplimient', 'establec', 'apart', '2', 'art..."


In [11]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])
stopwords.extend(['podemosen', 'podemen', 'ministro', 'ministra', 'ss', 'ruego', 'aprobado', 'aprobada'])
stopwords.extend(['diputado', 'diputada', 'don', 'doña', 'vicepresidenta', 'sagastizab', 'unzetabarrenetxea', 'agirretxea', 'urresti', 'escrutinio', 'dijo', 'dice', 'quevedo'])
stopwords.extend(['campuzano', 'beitialarrangoitia', 'gonzalezmoro', 'mixto'])

In [12]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.Nombre.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))

In [13]:
stopwords.extend(nombres)

In [15]:
partidos=pd.read_csv('/content/partidos.csv', encoding='latin1', sep=';')

In [16]:
for x in partidos['nombre'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(str(a).split())
  stopwords.extend(a)

for x in partidos['1'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['2'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['3'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

In [17]:
len(stopwords)

2829

In [18]:
base.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens'],
      dtype='object')

In [19]:
# elimino acentos de los tokens
a,b = 'áàéèíóúü','aaeeiouu'
trans = str.maketrans(a,b)
lista = []
for row in base.tokens:
    row=row.translate(trans)
    lista.append(row)

base['tokens']=lista

In [20]:
# llevo las stopwords a minúsculas y elimino acentos
stopwords=[x.lower() for x in stopwords if type(x)==str]
stopwords=[x.translate(trans) for x in stopwords if type(x)==str]

# Mencionadas en al menos 10 discursos
(spoken in at least 10 unique speaker-session)

In [21]:
base['tokens']

0                                                  ['abre']
1         ['cumplimient', 'establec', 'apart', '2', 'art...
2         ['letr', 'mayor', 'formacion', 'mes', 'edad', ...
3         ['acuerd', 'articul', '3', 'reglament', 'sen',...
4         ['señor', 'proced', 'constitu', 'sen', 'acuerd...
                                ...                        
156501    ['bo', 'dia', 'tod', 'senador', 'nacional', 'm...
156502    ['molt', 'bon', 'dia', 'bon', 'dia', 'senyor',...
156503    ['buen', 'dias', 'señoriassin', 'dud', 'encont...
156504    ['v', 'oy', 'termin', 'potenci', 'merc', 'post...
156505    ['señor', 'interveng', 'nombr', 'popul', 'fij'...
Name: tokens, Length: 156506, dtype: object

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords , min_df=10)
bow = vect.fit_transform(base['tokens'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['000001', '000018', '000260', '000261', '000284', '000285', '000289', '000293', '000294', '000296', '000297', '000298', '000301', '000302', '000303', '000305', '000306', '000307', '000308', '000311', '000314', '500', '542', '5oo', 'alvarez', 'azul', 'bo', 'cascos', 'compromis', 'congr', 'escudero', 'expediente504', 'gallardon', 'generales', 'hernandez', 'jeltzalea', 'letin', 'llorca', 'muñoz', 'na', 'oliva', 'ooo279', 'protestas', 'puirsir', 'soro'] not in stop_words.
  warnings.warn(


In [23]:
total_features = len(vect.vocabulary_)

# Mencionadas al menos 100 veces en total
(spoken at least 100 times across al sessions)

In [24]:
new_voc={}
for token in vect.vocabulary_:
  if vect.vocabulary_[token]>100:
    new_voc[token]=vect.vocabulary_[token]

In [25]:
len(new_voc)

244179

# Mencionadas al menos 10 veces en un día
(spoken at least 10 times in at least one session)

In [26]:
# Gentzkow et.al. usan mencionadas al menos 10 veces en una "sesión". Acá se usa mencionadas al menos 10 veces en un "día".

new_voc2={}
vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords)
for x in base.fecha_formato.unique():
  base_dia=base[pd.DatetimeIndex(base.fecha_formato)==x]
  try:
    bow = vect.fit_transform(base_dia['tokens'])
    for token in vect.vocabulary_:
      if vect.vocabulary_[token]>10:
        new_voc2[token]=vect.vocabulary_[token]
  except: pass

In [27]:
len(new_voc2)

5595823

# Combinación

In [28]:
inter=set(new_voc2).intersection(new_voc)

In [29]:
vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords ,vocabulary=inter)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

In [30]:
# La intersección genera un vocabulario de apenas 270230 bigramas.
total_features

244167

In [31]:
matrix = bow

In [33]:
import pickle

with open('vocabulary_senado_snowball.voc','wb') as f:
   pickle.dump(inter, f)